<div style="background-color: #fff8e1; border-left: 5px solid #ffc107; padding: 10px; margin-bottom: 10px;">
  <b>注意:</b> CloudWatch で Transaction Search を既に有効にしている場合は、このステップをスキップできます。
</div>

# Amazon Bedrock AgentCore Observability: Transaction Search の有効化
このノートブックでは、Amazon Bedrock AgentCore の可観測性を強化するために Transaction Search を有効にする方法を説明します。[Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/CloudWatch-Transaction-Search.html) は、アプリケーションのトランザクションスパンを完全に可視化できるインタラクティブな分析機能です。スパンは分散トレースの基本単位であり、アプリケーションやシステム内の特定のアクションやタスクを表します。各スパンは、トランザクションの特定のセグメントに関する詳細を記録します。これらの詳細には、開始時刻と終了時刻、期間、および顧客 ID や注文 ID などのビジネス属性を含むメタデータが含まれます。スパンは親子階層で配置されます。この階層は完全なトレースを形成し、異なるコンポーネントやサービス間でのトランザクションの流れをマッピングします。

Transaction Search は、コンソール、API、または AWS CloudFormation を使用して有効にできます。Transaction Search はアカウント全体で設定され、X-Ray を通じて取り込まれるすべてのスパンを [Amazon CloudWatch の料金](https://aws.amazon.com/cloudwatch/pricing/)に基づくコスト効率の高い収集モードに切り替えます。デフォルトでは、取り込まれたスパンの 1% を分析用のトレースサマリーとして無料でインデックス化します。これは、Transaction Search を通じてすべての取り込まれたスパンに対してエンドツーエンドのトレース可視性が既にあることを考えると、通常は十分です。

# 学習内容
- AWS CloudFormation を使用して Transaction Search を有効にする方法

# 前提条件
- [ドキュメント](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_update-role-permissions.html)に従って、CLI コマンドの実行、CloudFormation スタックの作成、Transaction Search の有効化に必要な権限を「SageMaker 実行ロール」に付与してください

# 1. セットアップと設定
CloudFormation を使用して Transaction Search を有効にするには、以下の 2 つのリソースを作成する必要があります。

- AWS::Logs::ResourcePolicy
- AWS::XRay::TransactionSearchConfig

サンプル [CloudFormation テンプレート](transaction_search.yml) がプロジェクトフォルダに提供されています。必要に応じて確認・更新してください。以下のコードを実行して、リソースポリシーを作成し、X-Ray が CloudWatch Logs にトレースを送信できるようにする Transaction Search を有効にします。

In [ ]:
!aws cloudformation create-stack --stack-name transaction-search --template-body file://transaction_search.yml

Transaction Search の有効化には 5 〜 10 分かかります。スタックの作成が `CREATE_COMPLETE` ステータスで完了するまで待ってください。以下のコードでステータスを確認できます。

In [ ]:
!aws cloudformation describe-stacks --stack-name transaction-search --query 'Stacks[0].StackStatus' --output text


# 2. 設定の確認

AWS CloudFormation スタックをデプロイした後、以下のコードで設定を確認できます。設定が成功していれば、Destination が「CloudWatchLogs」、Status が「ACTIVE」と表示されます。


In [ ]:
!aws xray get-trace-segment-destination


CloudWatch コンソールからも以下のように確認できます:

- Transaction Search 有効化前:

<div style="text-align:left">
    <img src="images/transaction_search_disabled1.png" width="100%"/>
</div>

<div style="text-align:left">
    <img src="images/transaction_search_disabled2.png" width="100%"/>
</div>

- Transaction Search 有効化後:

<div style="text-align:left">
    <img src="images/transaction_search_enabled1.png" width="100%"/>
</div>

<div style="text-align:left">
    <img src="images/transaction_search_enabled2.png" width="100%"/>
</div>



# 3. まとめ

Transaction Search を有効にすると、Application Signals や CloudWatch Logs の機能を含む複数の機能が利用可能になります。X-Ray に送信されたスパンは、aws/spans というロググループに取り込まれます。CloudWatch はこれらのスパンを使用して、CloudWatch Application Signals でキュレートされたアプリケーションパフォーマンスモニタリング（APM）エクスペリエンスを生成します。これにより、スパンの検索と分析が可能になり、外れ値やパターン検出などの CloudWatch Logs 機能も利用できます。カスタムメトリクス抽出も使用できます。CloudWatch Application Signals は、アプリケーション、サービス、および依存関係の統一されたアプリケーション中心のビューを提供します。また、アプリケーションの健全性を監視およびトリアージするのにも役立ちます。

このモジュールでは、CloudFormation を使用して Transaction Search を有効にする方法について説明しました。オプションとして、この[ドキュメント](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html)に従って、コンソール、API、または CLI を使用して Transaction Search を有効にすることもできます。

